#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2022, 3, 23)

### Yesterday = Last business day

In [2]:
#yesterday = date(2022, 3, 18)
yesterday

datetime.date(2022, 3, 23)

### Restart and Run All Cells

In [3]:
cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

In [4]:
format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [5]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1114911,DCC,2022-03-22,2022-03-23,1,2.96,2.98,0.02,0.68%,1,1,3.00,2.98,"4,418,044",2022-03-23,2022-03-23,1


In [6]:
names = tmp["name"]
name = names.to_string(index=False)
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'DCC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,718,DCC,3.40,2.46,O,2.80,0.00,42.86,0.62,0.00,"45,000",-4,4,0,0,0,6L,SET


### End of Tables in the process

In [7]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date >= '2022-03-23' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [8]:
df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['diff'] = (df['to_price'] - df['fm_price'])
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
name,,,,,,,,,,,
MAKRO,2022-03-17,2022-03-23,41.50,40.00,"74,394,130",40.50,-3.61,B,SET,-1.50,-3.61%
STA,2022-03-17,2022-03-23,28.00,26.75,"44,418,668",27.25,-3.57,B,SET100,-1.25,-4.46%
SYNEX,2022-03-21,2022-03-23,25.25,26.25,"12,473,124",26.50,0.99,B,SET100,1.00,3.96%
TISCO,2022-03-17,2022-03-23,100.00,99.00,"18,958,102",99.75,-1.00,B,SET50,-1.00,-1.00%
TMT,2022-03-22,2022-03-23,11.90,11.70,"2,653,924",11.90,11.60,B,SET,-0.20,-1.68%


### Price change up or down more than 3%

In [9]:
#df['bdays'] = np.busday_count(df['fm_date'],df['to_date'])
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.percent) >= 3,
    ['percent','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
JASIF,4.81%,I,10.40,10.90,SET,2022-03-08,15,11.00,4.81
SYNEX,3.96%,B,25.25,26.25,SET100,2022-03-21,2,26.50,0.99
KCE,3.59%,I,62.75,65.00,SET50,2022-03-21,2,66.25,1.20
SINGER,3.02%,I,49.75,51.25,SET100,2022-03-22,1,52.50,50.50
DOHOME,-3.48%,I,23.00,22.20,SET100,2022-03-22,1,23.20,22.10
MAKRO,-3.61%,B,41.50,40.00,SET,2022-03-17,6,40.50,-3.61
STA,-4.46%,B,28.00,26.75,SET100,2022-03-17,6,27.25,-3.57
RCL,-7.32%,I,51.25,47.50,SET100,2022-03-21,2,50.00,-1.46


In [10]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

watch_list.sort_values(['percent'],ascending=[False]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(box_file)
watch_list.sort_values(['percent'],ascending=[False]).to_csv(one_file)

### Create daily-sales from sales

In [11]:
df[cols].head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
name,,,,,,,,,
MAKRO,2022-03-17,2022-03-23,41.50,40.00,"74,394,130",40.50,-3.61,-3.61%,B
STA,2022-03-17,2022-03-23,28.00,26.75,"44,418,668",27.25,-3.57,-4.46%,B
SYNEX,2022-03-21,2022-03-23,25.25,26.25,"12,473,124",26.50,0.99,3.96%,B
TISCO,2022-03-17,2022-03-23,100.00,99.00,"18,958,102",99.75,-1.00,-1.00%,B
TMT,2022-03-22,2022-03-23,11.90,11.70,"2,653,924",11.90,11.60,-1.68%,B


In [12]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file,header=None)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(box_file)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file)

### Create monitors from stocks

In [13]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.head()

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DCC,B


In [14]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
WHERE trade IN ("B","I","O","S")
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,7
1,S,13


### output_file = iCloud Drive on PC

In [15]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [16]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,7
1,S,13


In [17]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
MAKRO,B,SET
STA,B,SET100
SYNEX,B,SET100
TISCO,B,SET50
TMT,B,SET
AIMIRT,I,SET
ASP,I,SET
BCH,I,SET100
CPNCG,I,SET


In [18]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [19]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
DCC,B
MAKRO,B
RCL,B
STA,B
SYNEX,B
TISCO,B
TMT,B


In [20]:
buy_candidates.shape[0]

7

In [21]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("I","S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AIMIRT,S
ASP,S
BCH,S
CPNCG,S
DOHOME,S
HREIT,S
IP,S
JASIF,S
KCE,S


In [22]:
sell_candidates.shape[0]

13